# Чем лечиться в домашней аптечке

У нас всех дома хранится много лекарств, и хорошо бы иметь быстрый и удобный доступ к данным о том, сколько у нас чего и в каких случаях что нужно принимать. То есть данные, которые хранит наша база, -- это данные о названии и количестве медикаментов, их сроке годности, а также данные из инструкций.

Инструкции к лекарствам легко найти в Интернете, но поиск по каждому отдельному лекарству -- дело довольно долгое. А собрав базу и по одному разу загрузив в неё слегка переработанную инструкцию, можно гораздо быстрее находить, что в нашей аптечке есть от головной боли, например. Или какие лекарства в нашей аптечке начинаются на "Асп", если мы забыли название и помним только первые три буквы. Или какие у нас есть лекарства с определённым действующим веществом. А ещё можно контролировать просрочку.

### Как работает база

Добавление в базу -- файл с текстом инструкци + информация о количестве упаковок и сроке годности

Поиск: 

по форме выпуска, действующему веществу и показаниям -- полнотекстовый

по сроку годности -- по дате

по названию -- RegEx

Выдача сортируется по сроку годности (чем раньше истекает срок годности, тем выше препарат в выдаче)

Апдейт -- по названию медикамента. Апдейтится количество и срок годности

Удаление из базы -- по названию медикамента

In [464]:
import pandas as pd
import re
import datetime
import pymorphy2
import pymongo
from bson import ObjectId

## Подключение к базе

In [3]:
client = pymongo.MongoClient("mongodb+srv://projectowner:<password>@projectcluster-0aq6m.gcp.mongodb.net/test?retryWrites=true&w=majority")
db = client.test

In [4]:
db

Database(MongoClient(host=['projectcluster-shard-00-02-0aq6m.gcp.mongodb.net:27017', 'projectcluster-shard-00-00-0aq6m.gcp.mongodb.net:27017', 'projectcluster-shard-00-01-0aq6m.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ProjectCluster-shard-0', ssl=True), 'test')

In [5]:
coll_meds = db['medicine_instructions']

In [251]:
# coll_meds.create_index([("Форма выпуска", "text"), ("Действующее вещество", "text"), ("Показания", "text")], default_language='russian')

'Форма выпуска_text_Действующее вещество_text_Показания_text'

## Функции для добавления нового лекарства в базу

In [393]:
params = ['Годен до']
tries = 0

In [143]:
def parse_text(link):
    with open(link, 'r', encoding='utf-8') as text:
        text = text.read()
    text = re.sub('\n\n', '\n', text)
    headers = re.findall('(@.+?)\n', text)
    return text, headers

In [144]:
def enter_quantity():
    global tries
    tries = 0
    quantity = int(input('Количество упаковок: '))
    if quantity <= 0 and tries < 3: 
        tries += 1
        quantity = enter_quantity()
    elif tries == 3:
        raise BaseException
    return quantity

In [145]:
def enter_expiry():
    global tries
    tries = 0
    expiry = input('Срок годности в формате ГГГГ-ММ ')
    try:
        expiry_list = [int(item) for item in expiry.split('-')]
        expiry_py = datetime.date(expiry_list[0], expiry_list[1], 1)
        tries = 0
    except Exception:
        tries += 1
        print('Такой даты не существует!')
        if tries < 3:
            expiry = enter_expiry()
        else:
            tries = 0
            raise BaseException
    return expiry

In [199]:
def insert_data(link):
    """
    Функция добавляет вашу инструкцию в базу.
    Входные данные: link -- ссылка на файл с текстом инструкции.
    
    """
    global params
    text, headers = parse_text(link)
    for item in headers:
        item = item.strip('@ ')
        if item not in params:
            params.append(item)
    doc_dict = {}
    doc_list = text.split('\n')
    for i in range(len(doc_list)):
        if doc_list[i] in headers:
            val = ''
            for j in range(i+1, len(doc_list)):
                if doc_list[j] in headers:
                    break
                val = val + '\n' + doc_list[j]
            doc_dict[doc_list[i].strip('@ ')] = val.strip('\n')
    quantity = enter_quantity()
    doc_dict['Количество'] = quantity
    expiry = enter_expiry() + '-01'
    doc_dict['Годен до'] = 'new Date("{}")'.format(expiry)
    ins = coll_meds.insert_one(doc_dict)
    med_name = doc_dict['Название']
    print('Инструкция к препарату {} загружена успешно'.format(med_name))

In [147]:
def insert_instruction():
    print('Проверьте ваш файл. Заголовки пунктов инструкции должны быть отделены символом @ в начале и символом конца строки в конце.')
    status_req = 'Ваш текст инструкции оформлен правильно? Ответьте "да" или "нет".\n'
    status = input(status_req)
    if status.lower() == 'нет':
        print('Ничего, я подожду. Приходите ещё!')
    elif status.lower() == 'да':
        link = input('Ссылка на текст: ')
        insert_data(link)
    else:
        print('Что-то пошло не так, и мы вас не поняли :(')

## Функции для поиска по базе

In [483]:
def find():
    """
    Функция ищет документы по заданным параметрам.
    Возвращает выдачу из базы в формате датафрейма.
    
    """
    req = {}
    params = ["Название", "Форма выпуска", "Действующее вещество", "Показания", "Годен до"]
    
    print('Введите значения для каждого параметра. Если вам нужны все значения какого-то параметра, оставьте строку пустой.')
    for item in params:
        res = input(item + '\n')
        req[item] = res
    
    if req["Годен до"]:
        req['Годен до'] = 'new Date("{}")'.format(req["Годен до"])

    try:
        res_name = coll_meds.find({"Название": {"$regex": ".*().*".format(req["Название"])}}) if req["Название"] else coll_meds.find({})
        res_form = coll_meds.find({"$text": {"$search": req["Форма выпуска"]}}) if req["Форма выпуска"] else coll_meds.find({})
        if req["Показания"]:
            res_need = []
            for word in req["Показания"].split(' '):
                res_need.append(coll_meds.find({"$text": {"$search": word.strip()}}))
        else: 
            res_need = [coll_meds.find({})]
        res_subst = coll_meds.find({"$text": {"$search": req["Действующее вещество"]}}) if req["Действующее вещество"] else coll_meds.find({})
        res_nonexp = coll_meds.find({"Годен до": {"$gte": req["Годен до"]}}) if req["Годен до"] else coll_meds.find({})
        
        res_list = [res_name, res_form, res_subst, res_nonexp] + res_need
        for i in range(len(res_list)):
            res_list[i] = list(res_list[i].sort("Годен до", pymongo.ASCENDING))
        
#         res_name, res_form, res_need, res_subst, res_nonexp = res_list
        final = []
        for item in res_list[0]:
            count = 0
            for res in res_list[1:]:
                if item in res:
                    count += 1
            if count == len(res_list) - 1:
                final.append(item)
                                
#         final = [item for item in res_name if item in res_form and item in res_need and item in res_subst and item in res_nonexp]
        df = pd.DataFrame(final)
    #         for item in req:
#             if item not in ["Форма выпуска", "Показания", "Действующее вещество"] and req[item] != '$all':
#                 que[item] = req[item]
#         res_general = list(coll_meds.find(que))
        return df
    except Exception as e:
        print('Exception!', e)

In [434]:
def specific_meds(df):
    spec = {}
    req_name = input('Какой препарат вас интересует?\n')
    while req_name != '':
        col_name = input('Какие главы инструкции к препарату {} вас интересуют?'.format(req_name) + ' Перечислите через точку с запятой\n')
        spec[req_name] = col_name.split(';')
        req_name = input('Какой препарат вас интересует? Оставьте строку пустой, если закончили выбор\n')
    for item in spec:
        print(item)
        for col in spec[item]:
            print(col + ': \n' + str(df[df["Название"]==item][col].values[0]) + '\n\n')

## Функция для изменения данных в базе

In [453]:
def update_quantity():
    name = input('Название препарата: ')
    new_quantity = enter_quantity()
    new_expiry = enter_expiry()
    upd = coll_meds.update_one({"Название": name}, {"$set": {"Количество упаковок": new_quantity, "Годен до": 'new Date("{}-01")'.format(new_expiry)}})
    print('Данные обновлены')

## Функция для удаления данных из базы

In [306]:
def delete_medicine():
    med_name = input('Как называется препарат, который вы хотите удалить?\n')
    delone = coll_meds.delete_one({"Название": "{}".format(med_name)})
    print('Препарат удалён из базы')

## А вот так это работает

In [396]:
insert_instruction()

Проверьте ваш файл. Заголовки пунктов инструкции должны быть отделены символом @ в начале и символом конца строки в конце.
Ваш текст инструкции оформлен правильно? Ответьте "да" или "нет".
да
Ссылка на текст: phenotropil.txt
Количество упаковок: 1
Срок годности в формате ГГГГ-ММ 2022-02
Инструкция к препарату Фенотропил загружена успешно


In [468]:
search_result = find()
search_result

Введите значения для каждого параметра. Если вам нужны все значения какого-то параметра, оставьте строку пустой.
Название

Форма выпуска

Действующее вещество

Показания
топографический кретинизм
Годен до



""


In [486]:
search_result = find()
search_result

Введите значения для каждого параметра. Если вам нужны все значения какого-то параметра, оставьте строку пустой.
Название

Форма выпуска

Действующее вещество

Показания
зубная боль
Годен до



,_id,Название,Форма выпуска,Действующее вещество,Вспомогательные вещества,Упаковка,Фармакологическое действие,Фармакокинетика,Показания,Способ применения и дозы,Побочные действия,Противопоказания,Применение,Особые указания,Взаимодействие,Количество,Годен до,Количество упаковок
0,5e89aba7b7d260d854f06847,Нурофен,"Таблетки, покрытые оболочкой белого или почти ...",1 таб.\nибупрофен - 200 мг,"кроскармеллоза натрия - 30 мг, натрия лаурилсу...",6 шт. - блистеры (1) - пачки картонные.\n6 шт....,"НПВС, производное фенилпропионовой кислоты. Ок...",При приеме внутрь ибупрофен практически полнос...,Для системного применения\nСимптоматическое ле...,"Устанавливают индивидуально, в зависимости от ...",Со стороны системы кроветворения: очень редко ...,Повышенная чувствительность к ибупрофену.\nДля...,Применение при беременности и кормлении грудью...,Рекомендуется назначать ибупрофен максимально ...,При одновременном применении ибупрофен уменьша...,3,"new Date(""2022-10-01"")",NaN
1,5e89abe0b7d260d854f06848,Фуфлофен,"Таблетки, покрытые оболочкой белого или почти ...",1 таб.\nфуфлосемид - 200 мг,NaN,NaN,"НПВС, производное фенилпропионовой кислоты. Ок...",При приеме внутрь ибупрофен практически полнос...,Для системного применения\nСимптоматическое ле...,"Устанавливают индивидуально, в зависимости от ...",Со стороны системы кроветворения: очень редко ...,Повышенная чувствительность к ибупрофену.\nДля...,Применение при беременности и кормлении грудью...,Рекомендуется назначать ибупрофен максимально ...,При одновременном применении ибупрофен уменьша...,4,"new Date(""2023-01-01"")",4.0


In [426]:
specific_meds(search_result)

Какой препарат вас интересует?
Нурофен
Какие главы инструкции к препарату Нурофен вас интересуют? Перечислите через точку с запятой
Фармакокинетика;Противопоказания
Какой препарат вас интересует? Оставьте строку пустой, если закончили выбор
Фуфлофен
Какие главы инструкции к препарату Фуфлофен вас интересуют? Перечислите через точку с запятой
Действующее вещество
Какой препарат вас интересует? Оставьте строку пустой, если закончили выбор

Нурофен
Фармакокинетика: 
При приеме внутрь ибупрофен практически полностью абсорбируется из ЖКТ. Одновременный прием пищи замедляет скорость всасывания. Метаболизируется в печени (90%). T1/2 составляет 2-3 ч.
После нанесения на кожу ибупрофен обнаруживается в эпидермисе и дерме через 24 ч. Достигает высокой терапевтической концентрации в подлежащих мягких тканях, суставах и синовиальной жидкости. Клинически значимого системного всасывания практически не происходит. Cmax ибупрофена в плазме крови при наружном применении составляет 5% от уровня Cmax при

In [454]:
update_quantity()

Название препарата: Фуфлофен
Количество упаковок: 4
Срок годности в формате ГГГГ-ММ 2023-01
Данные обновлены


In [333]:
delete_medicine()

Как называется препарат, который вы хотите удалить?
Нурофен
Препарат удалён из базы


In [390]:
print(coll_meds.find_one({"Название": "Нурофен"})["Противопоказания"])

TypeError: 'NoneType' object is not subscriptable